In [1]:
#importing census data and census tracts data
import pandas as pd

directory = "../raw_data_lite/"


census_2018 = pd.read_csv(f"{directory}ACSDP5Y2018.DP03_data_with_overlays_2021-08-03T221820.csv")
census_2017 = pd.read_csv(f"{directory}ACSDP5Y2017.DP03_data_with_overlays_2021-08-10T212233.csv")


/home/binhducvu/jupyterenv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (2,3,4,6,7,10,11,14,15,18,19,22,23,26,27,30,31,32,38,39,40,42,43,46,47,50,51,54,55,56,58,59,62,63,64,66,67,70,71,72,74,75,78,79,82,83,86,87,90,91,94,95,102,103,104,106,107,110,111,114,115,118,119,122,123,126,127,128,130,131,134,135,138,139,142,143,146,147,150,151,154,155,158,159,162,163,166,167,170,171,174,175,178,179,182,183,184,186,187,190,191,194,195,198,199,202,203,204,206,207,210,211,214,215,218,219,222,223,226,227,230,231,234,235,238,239,242,243,254,255,262,263,270,271,278,279,286,287,294,295,298,299,300,302,303,306,307,310,311,314,315,318,319,322,323,326,327,330,331,334,335,338,339,354,355,356,378,379,380,382,383,386,387,390,391,394,395,398,399,400,402,403,406,407,408,410,411,412,414,415,416,418,419,422,423,426,427,430,431,434,435,436,438,439,442,443,446,447,450,451,454,455,456,458,459,462,463,466,467,470,471) have mixed types.Specify dtype option on import or set 

# Preprocessing the data


In [2]:
#getting all the necessary geojson files

import geopandas as gpd
#converting the tracts and taxi zones data into geojson files
directory = "../raw_data_lite/"
taxi_zones_shp = gpd.read_file(f"{directory}taxi_zones.shp")
taxi_zones_shp.to_file(f"{directory}taxi_zones.geojson", drivers = 'GeoJSON')

tracts_shp = gpd.read_file(f"{directory}nyct2010_21b/nyct2010.shp")
tracts_shp.to_file(f"{directory}tracts.geojson", drivers = 'GeoJSON')
#reading the files in as dataframes
taxi_zones = gpd.read_file(f"{directory}taxi_zones.geojson")
tracts = gpd.read_file(f"{directory}tracts.geojson")

/home/binhducvu/jupyterenv/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
import numpy as np

#replacing all unavailable data with np.NaN for type consistency
census_2017.replace('(X)', np.nan)
census_2018.replace('(X)', np.nan)

,GEO_ID,NAME,DP03_0001E,DP03_0001M,DP03_0001PE,DP03_0001PM,DP03_0002E,DP03_0002M,DP03_0002PE,DP03_0002PM,...,DP03_0135PE,DP03_0135PM,DP03_0136E,DP03_0136M,DP03_0136PE,DP03_0136PM,DP03_0137E,DP03_0137M,DP03_0137PE,DP03_0137PM
0,id,Geographic Area Name,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...,Margin of Error!!EMPLOYMENT STATUS!!Population...,Percent Estimate!!EMPLOYMENT STATUS!!Populatio...,Percent Margin of Error!!EMPLOYMENT STATUS!!Po...,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...,Margin of Error!!EMPLOYMENT STATUS!!Population...,Percent Estimate!!EMPLOYMENT STATUS!!Populatio...,Percent Margin of Error!!EMPLOYMENT STATUS!!Po...,...,Percent Estimate!!PERCENTAGE OF FAMILIES AND P...,Percent Margin of Error!!PERCENTAGE OF FAMILIE...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Margin of Error!!PERCENTAGE OF FAMILIES AND PE...,Percent Estimate!!PERCENTAGE OF FAMILIES AND P...,Percent Margin of Error!!PERCENTAGE OF FAMILIE...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Margin of Error!!PERCENTAGE OF FAMILIES AND PE...,Percent Estimate!!PERCENTAGE OF FAMILIES AND P...,Percent Margin of Error!!PERCENTAGE OF FAMILIE...
1,1400000US36005000100,"Census Tract 1, Bronx County, New York",7080,290,7080,NaN,0,17,0.0,0.5,...,-,**,NaN,NaN,-,**,NaN,NaN,-,**
2,1400000US36005000200,"Census Tract 2, Bronx County, New York",3684,491,3684,NaN,1873,296,50.8,6.7,...,12.5,7.3,NaN,NaN,16.2,10.0,NaN,NaN,48.8,11.3
3,1400000US36005000400,"Census Tract 4, Bronx County, New York",4689,425,4689,NaN,3052,385,65.1,5.1,...,14.4,9.3,NaN,NaN,7.3,4.7,NaN,NaN,21.5,10.9
4,1400000US36005001600,"Census Tract 16, Bronx County, New York",4598,351,4598,NaN,2477,266,53.9,4.5,...,22.9,9.7,NaN,NaN,21.2,8.7,NaN,NaN,28.9,10.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2163,1400000US36085030302,"Census Tract 303.02, Richmond County, New York",4852,447,4852,NaN,2961,367,61.0,6.1,...,7.7,9.3,NaN,NaN,17.2,7.4,NaN,NaN,8.2,6.7
2164,1400000US36085031901,"Census Tract 319.01, Richmond County, New York",2107,322,2107,NaN,818,245,38.8,7.9,...,60.7,18.9,NaN,NaN,41.1,12.6,NaN,NaN,89.4,8.7
2165,1400000US36085031902,"Census Tract 319.02, Richmond County, New York",3716,379,3716,NaN,1946,339,52.4,6.9,...,43.5,20.8,NaN,NaN,33.8,12.2,NaN,NaN,58.7,17.6
2166,1400000US36085032300,"Census Tract 323, Richmond County, New York",939,152,939,NaN,527,141,56.1,12.3,...,0.0,26.1,NaN,NaN,19.4,15.5,NaN,NaN,38.6,24.4


In [4]:
#Forming a header and their respective IDs from columns of interest, done by inspection
# of original dataset - no mapping guide available. All estimates and margin of errors
# are taken in place of percent estimates as we need to aggregate certain tracts

header = ['GEO_ID', 'Name', 
         'Civilian labor force estimates', 
         'Unemployment estimates',  
         'Total households estimates', 
         'Less than 10,000 estimates', 
         '10,000 to 14,999 estimates', 
         '15,000 to 24,999 estimates',
         '25,000 to 34,999 estimates',
         '35,000 to 49,999 estimates',
         '50,000 to 74,999 estimates',
         '75,000 to 99,999 estimates',
         '100,000 to 149,999 estimates', 
         '150,000 to 199,999 estimates', 
         '200,000 or more estimates', ]

header_ID = ['GEO_ID', 'NAME',
            'DP03_0003E', 'DP03_0005E',
            'DP03_0051E', 'DP03_0052E', 
            'DP03_0053E', 'DP03_0054E', 
            'DP03_0055E', 'DP03_0056E',
            'DP03_0057E', 'DP03_0058E',
            'DP03_0059E', 'DP03_0060E', 
            'DP03_0061E' ]

# create dictionary that assigns the IDs into readable names
header_dict = {header_ID[i]: header[i] for i in range(0, len(header))}
    
header_dict

{'GEO_ID': 'GEO_ID',
 'NAME': 'Name',
 'DP03_0003E': 'Civilian labor force estimates',
 'DP03_0005E': 'Unemployment estimates',
 'DP03_0051E': 'Total households estimates',
 'DP03_0052E': 'Less than 10,000 estimates',
 'DP03_0053E': '10,000 to 14,999 estimates',
 'DP03_0054E': '15,000 to 24,999 estimates',
 'DP03_0055E': '25,000 to 34,999 estimates',
 'DP03_0056E': '35,000 to 49,999 estimates',
 'DP03_0057E': '50,000 to 74,999 estimates',
 'DP03_0058E': '75,000 to 99,999 estimates',
 'DP03_0059E': '100,000 to 149,999 estimates',
 'DP03_0060E': '150,000 to 199,999 estimates',
 'DP03_0061E': '200,000 or more estimates'}

In [5]:
#formatting the dataset
def census_converter(dataset):
    """A function that takes the census dataset and returns wanted columns"""
    new_dataset = dataset[header_ID]

    return new_dataset
#converting each set into one with correct headers and columns of interest
census_2017 = census_converter(census_2017)
census_2017 = census_2017.iloc[1:,:]
census_2017.rename(columns = header_dict, inplace = True)

census_2018 = census_converter(census_2018)
census_2018 = census_2018.iloc[1:,:]
census_2018.rename(columns = header_dict, inplace = True)


In [6]:
#save the preprocessed census files into preprocessed_data

census_2018.to_csv('../preprocessed_data/census_2018.csv')
census_2017.to_csv('../preprocessed_data/census_2017.csv')

In [7]:
census_2018

,GEO_ID,Name,Civilian labor force estimates,Unemployment estimates,Total households estimates,"Less than 10,000 estimates","10,000 to 14,999 estimates","15,000 to 24,999 estimates","25,000 to 34,999 estimates","35,000 to 49,999 estimates","50,000 to 74,999 estimates","75,000 to 99,999 estimates","100,000 to 149,999 estimates","150,000 to 199,999 estimates","200,000 or more estimates"
1,1400000US36005000100,"Census Tract 1, Bronx County, New York",0,0,0,0,0,0,0,0,0,0,0,0,0
2,1400000US36005000200,"Census Tract 2, Bronx County, New York",1873,295,1328,91,32,163,131,153,271,126,204,79,78
3,1400000US36005000400,"Census Tract 4, Bronx County, New York",3052,244,1963,142,38,149,68,166,353,337,340,247,123
4,1400000US36005001600,"Census Tract 16, Bronx County, New York",2477,164,1982,167,247,233,327,299,352,161,152,23,21
5,1400000US36005001900,"Census Tract 19, Bronx County, New York",1451,192,929,127,96,62,67,169,179,115,99,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2163,1400000US36085030302,"Census Tract 303.02, Richmond County, New York",2961,123,1964,140,52,98,67,125,389,378,436,230,49
2164,1400000US36085031901,"Census Tract 319.01, Richmond County, New York",818,71,1055,529,91,40,30,48,133,100,68,7,9
2165,1400000US36085031902,"Census Tract 319.02, Richmond County, New York",1946,72,1670,482,86,166,67,173,102,91,278,148,77
2166,1400000US36085032300,"Census Tract 323, Richmond County, New York",527,10,440,122,0,37,8,18,44,79,43,83,6


In [8]:
census_2017

,GEO_ID,Name,Civilian labor force estimates,Unemployment estimates,Total households estimates,"Less than 10,000 estimates","10,000 to 14,999 estimates","15,000 to 24,999 estimates","25,000 to 34,999 estimates","35,000 to 49,999 estimates","50,000 to 74,999 estimates","75,000 to 99,999 estimates","100,000 to 149,999 estimates","150,000 to 199,999 estimates","200,000 or more estimates"
1,1400000US36061000100,"Census Tract 1, New York County, New York",0,0,0,0,0,0,0,0,0,0,0,0,0
2,1400000US36061000201,"Census Tract 2.01, New York County, New York",1014,27,985,254,121,154,190,175,55,10,11,8,7
3,1400000US36061000202,"Census Tract 2.02, New York County, New York",3229,542,3434,576,314,526,407,375,479,131,284,167,175
4,1400000US36061000500,"Census Tract 5, New York County, New York",0,0,0,0,0,0,0,0,0,0,0,0,0
5,1400000US36061000600,"Census Tract 6, New York County, New York",4031,502,4098,823,913,718,273,497,473,105,218,0,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2163,1400000US36085030302,"Census Tract 303.02, Richmond County, New York",2834,113,1985,172,17,124,117,120,379,387,389,229,51
2164,1400000US36085031901,"Census Tract 319.01, Richmond County, New York",743,60,1057,541,89,46,49,7,150,93,66,0,16
2165,1400000US36085031902,"Census Tract 319.02, Richmond County, New York",1797,83,1711,566,101,135,28,155,132,100,338,108,48
2166,1400000US36085032300,"Census Tract 323, Richmond County, New York",572,28,444,115,5,20,8,27,57,86,49,73,4
